# Update Metadata

In [5]:
from resdb_driver.crypto import generate_keypair
from resdb_driver import Resdb

db_root_url = "http://0.0.0.0:18000"

db = Resdb(db_root_url)

alice, bob = generate_keypair(), generate_keypair()

# create a digital asset for Alice
game_boy_token = {
    "data": {
        "token_for": {"game_boy": {"serial_number": "LR35902"}},
        "description": "Time share token. Each token equals one hour of usage.",
    },
}

# prepare the transaction with the digital asset and issue 10 tokens for Bob
prepared_token_tx = db.transactions.prepare(
    operation="CREATE",
    signers=alice.public_key,
    recipients=[([bob.public_key], 10)],
    asset=game_boy_token,
    metadata={"data":"DODO"}
)

# fulfill the tnx
fulfilled_token_tx = db.transactions.fulfill(
    prepared_token_tx, private_keys=alice.private_key)

HASH DATA <class 'str'>, data='{"asset":{"data":{"description":"Time share token. Each token equals one hour of usage.","token_for":{"game_boy":{"serial_number":"LR35902"}}}},"id":null,"inputs":[{"fulfillment":"pGSAINO79V-DFNObaEkLoIym5lVFGWmcPknbQt-dy4zhRhCdgUDUdstEeLlea4aB_it_7dw_GRpT9Sxsqxo4GWg31Q7z5ZHKrCinJOYtWYB7sPSKyPYBFj0GRgHwQcC-KV9te0AG","fulfills":null,"owners_before":["FFXFq4rpq1CmW61qyRsADdhZK19PgjF49uTUkPqKET8x"]}],"metadata":{"data":"DODO"},"operation":"CREATE","outputs":[{"amount":"10","condition":{"details":{"public_key":"94D84hkLng4cfpCG51jksUqHRfYByhhtEUXxBoT7TcVW","type":"ed25519-sha-256"},"uri":"ni:///sha-256;1tqZ3kY8z0YqUfcg_dUFnCVZSJpOcBkV-lEXzyWui6c?fpt=ed25519-sha-256&cost=131072"},"public_keys":["94D84hkLng4cfpCG51jksUqHRfYByhhtEUXxBoT7TcVW"]}],"version":"2.0"}'


In [6]:
fulfilled_token_tx


{'inputs': [{'owners_before': ['FFXFq4rpq1CmW61qyRsADdhZK19PgjF49uTUkPqKET8x'],
   'fulfills': None,
   'fulfillment': 'pGSAINO79V-DFNObaEkLoIym5lVFGWmcPknbQt-dy4zhRhCdgUDUdstEeLlea4aB_it_7dw_GRpT9Sxsqxo4GWg31Q7z5ZHKrCinJOYtWYB7sPSKyPYBFj0GRgHwQcC-KV9te0AG'}],
 'outputs': [{'public_keys': ['94D84hkLng4cfpCG51jksUqHRfYByhhtEUXxBoT7TcVW'],
   'condition': {'details': {'type': 'ed25519-sha-256',
     'public_key': '94D84hkLng4cfpCG51jksUqHRfYByhhtEUXxBoT7TcVW'},
    'uri': 'ni:///sha-256;1tqZ3kY8z0YqUfcg_dUFnCVZSJpOcBkV-lEXzyWui6c?fpt=ed25519-sha-256&cost=131072'},
   'amount': '10'}],
 'operation': 'CREATE',
 'metadata': {'data': 'DODO'},
 'asset': {'data': {'token_for': {'game_boy': {'serial_number': 'LR35902'}},
   'description': 'Time share token. Each token equals one hour of usage.'}},
 'version': '2.0',
 'id': '0212ad78049e42b7d280c66391066824e50f654d4de9d9bc235264422b9622a5'}

In [3]:
tx_reciept = db.transactions.send_commit(fulfilled_token_tx)
tx_reciept_id = tx_reciept[4:]

In [4]:
tx_reciept_id


'c9f95dd17cd7874e6d2c4e933e8b276f4e3d98e6b6b55be98c786ef8aa9e262c'

In [5]:
create_tx_info = db.transactions.retrieve(tx_reciept_id)
create_tx_info

{'inputs': [{'owners_before': ['HVevXcmRWDSMsUgy62maJfNLgdEki2KG9KbzVQKxyx2d'],
   'fulfills': None,
   'fulfillment': 'pGSAIPUSIg4x_4mAXs3P1cAwPyWMfgDkfVfOqV1lP4xBLlpKgUCjvi536HzolOtDRj3U7byjdcDrSlSan0wsIfDsWji8PshPCF5VrPUL_QhB_Q1QXl-JR5HZ9FwF0mtNVLwK_zoA'}],
 'outputs': [{'public_keys': ['HVevXcmRWDSMsUgy62maJfNLgdEki2KG9KbzVQKxyx2d'],
   'condition': {'details': {'type': 'ed25519-sha-256',
     'public_key': 'HVevXcmRWDSMsUgy62maJfNLgdEki2KG9KbzVQKxyx2d'},
    'uri': 'ni:///sha-256;HRikWGBifo0VSEXQhrnQDBGvxr54SB9rbhU3VjGgwEs?fpt=ed25519-sha-256&cost=131072'},
   'amount': '1'}],
 'operation': 'CREATE',
 'metadata': {'data': 'INIT'},
 'asset': {'data': {'token_for': {'game_boy': {'serial_number': 'LR35902'}},
   'description': 'Time share token. Each token equals one hour of usage.'}},
 'version': '2.0',
 'id': 'c9f95dd17cd7874e6d2c4e933e8b276f4e3d98e6b6b55be98c786ef8aa9e262c'}

In [6]:
transfer_asset = {"id": fulfilled_token_tx["id"]}
output_index = 0
output = fulfilled_token_tx["outputs"][output_index]
transfer_input = {
    "fulfillment": output["condition"]["details"],
    "fulfills": {"output_index": output_index, "transaction_id": transfer_asset["id"]},
    "owners_before": output["public_keys"],
}

In [32]:
transfer_input

{'fulfillment': {'type': 'ed25519-sha-256',
  'public_key': 'CfEpmfVLny134UvWczVux4j8td9srvyeYy7hPEZXT1jG'},
 'fulfills': {'output_index': 0,
  'transaction_id': '73a6e5517037673ca1bc49dc6df9c722b425e0e3f4cea166a143596260e16ce2'},
 'owners_before': ['CfEpmfVLny134UvWczVux4j8td9srvyeYy7hPEZXT1jG']}

In [33]:
prepared_transfer_tx = db.transactions.prepare(
    operation="TRANSFER",
    asset=transfer_asset,
    inputs=transfer_input,
    metadata={"data":"2323"},
    recipients=[(
                [bob.public_key],
                 10
                 )],
)

In [34]:
prepared_transfer_tx

{'inputs': [{'owners_before': ['CfEpmfVLny134UvWczVux4j8td9srvyeYy7hPEZXT1jG'],
   'fulfills': {'transaction_id': '73a6e5517037673ca1bc49dc6df9c722b425e0e3f4cea166a143596260e16ce2',
    'output_index': 0},
   'fulfillment': {'type': 'ed25519-sha-256',
    'public_key': 'CfEpmfVLny134UvWczVux4j8td9srvyeYy7hPEZXT1jG'}}],
 'outputs': [{'public_keys': ['CfEpmfVLny134UvWczVux4j8td9srvyeYy7hPEZXT1jG'],
   'condition': {'details': {'type': 'ed25519-sha-256',
     'public_key': 'CfEpmfVLny134UvWczVux4j8td9srvyeYy7hPEZXT1jG'},
    'uri': 'ni:///sha-256;53p3B2t3tovqxEsX6x0rdPgJ1Rw7OFDE47EdTYf8fVg?fpt=ed25519-sha-256&cost=131072'},
   'amount': '10'}],
 'operation': 'TRANSFER',
 'metadata': {'data': '2323'},
 'asset': {'id': '73a6e5517037673ca1bc49dc6df9c722b425e0e3f4cea166a143596260e16ce2'},
 'version': '2.0',
 'id': None}

In [35]:
fulfilled_transfer_tx = db.transactions.fulfill(
    prepared_transfer_tx,
    private_keys=bob.private_key
)

HASH DATA <class 'str'>, data='{"asset":{"id":"73a6e5517037673ca1bc49dc6df9c722b425e0e3f4cea166a143596260e16ce2"},"id":null,"inputs":[{"fulfillment":"pGSAIK08QeHuTDKvTolIFw2RSZ2K0PFFI7F2JVF1zbjVXNHDgUBZh4KTnCv8mYnvXpzSqFWASt7mQ6nfL6Dyy9XmfC_Ekcl6_gR738EP2CJdF094HbhB2xlmhSE58XojsdEvGoIN","fulfills":{"output_index":0,"transaction_id":"73a6e5517037673ca1bc49dc6df9c722b425e0e3f4cea166a143596260e16ce2"},"owners_before":["CfEpmfVLny134UvWczVux4j8td9srvyeYy7hPEZXT1jG"]}],"metadata":{"data":"2323"},"operation":"TRANSFER","outputs":[{"amount":"10","condition":{"details":{"public_key":"CfEpmfVLny134UvWczVux4j8td9srvyeYy7hPEZXT1jG","type":"ed25519-sha-256"},"uri":"ni:///sha-256;53p3B2t3tovqxEsX6x0rdPgJ1Rw7OFDE47EdTYf8fVg?fpt=ed25519-sha-256&cost=131072"},"public_keys":["CfEpmfVLny134UvWczVux4j8td9srvyeYy7hPEZXT1jG"]}],"version":"2.0"}'


In [36]:
fulfilled_transfer_tx

{'inputs': [{'owners_before': ['CfEpmfVLny134UvWczVux4j8td9srvyeYy7hPEZXT1jG'],
   'fulfills': {'transaction_id': '73a6e5517037673ca1bc49dc6df9c722b425e0e3f4cea166a143596260e16ce2',
    'output_index': 0},
   'fulfillment': 'pGSAIK08QeHuTDKvTolIFw2RSZ2K0PFFI7F2JVF1zbjVXNHDgUBZh4KTnCv8mYnvXpzSqFWASt7mQ6nfL6Dyy9XmfC_Ekcl6_gR738EP2CJdF094HbhB2xlmhSE58XojsdEvGoIN'}],
 'outputs': [{'public_keys': ['CfEpmfVLny134UvWczVux4j8td9srvyeYy7hPEZXT1jG'],
   'condition': {'details': {'type': 'ed25519-sha-256',
     'public_key': 'CfEpmfVLny134UvWczVux4j8td9srvyeYy7hPEZXT1jG'},
    'uri': 'ni:///sha-256;53p3B2t3tovqxEsX6x0rdPgJ1Rw7OFDE47EdTYf8fVg?fpt=ed25519-sha-256&cost=131072'},
   'amount': '10'}],
 'operation': 'TRANSFER',
 'metadata': {'data': '2323'},
 'asset': {'id': '73a6e5517037673ca1bc49dc6df9c722b425e0e3f4cea166a143596260e16ce2'},
 'version': '2.0',
 'id': 'e80cfb8241f45370d8df6caad9f531e2a3c061d7c816b4154f72f5429153441d'}

In [37]:
transfer_tx = db.transactions.send_commit(fulfilled_transfer_tx)
transfer_tx_id = transfer_tx[4:]

In [38]:
transfer_tx_info = db.transactions.retrieve(transfer_tx_id)
transfer_tx_info

{'inputs': [{'owners_before': ['CfEpmfVLny134UvWczVux4j8td9srvyeYy7hPEZXT1jG'],
   'fulfills': {'transaction_id': '73a6e5517037673ca1bc49dc6df9c722b425e0e3f4cea166a143596260e16ce2',
    'output_index': 0},
   'fulfillment': 'pGSAIK08QeHuTDKvTolIFw2RSZ2K0PFFI7F2JVF1zbjVXNHDgUBZh4KTnCv8mYnvXpzSqFWASt7mQ6nfL6Dyy9XmfC_Ekcl6_gR738EP2CJdF094HbhB2xlmhSE58XojsdEvGoIN'}],
 'outputs': [{'public_keys': ['CfEpmfVLny134UvWczVux4j8td9srvyeYy7hPEZXT1jG'],
   'condition': {'details': {'type': 'ed25519-sha-256',
     'public_key': 'CfEpmfVLny134UvWczVux4j8td9srvyeYy7hPEZXT1jG'},
    'uri': 'ni:///sha-256;53p3B2t3tovqxEsX6x0rdPgJ1Rw7OFDE47EdTYf8fVg?fpt=ed25519-sha-256&cost=131072'},
   'amount': '10'}],
 'operation': 'TRANSFER',
 'metadata': {'data': '2323'},
 'asset': {'id': '73a6e5517037673ca1bc49dc6df9c722b425e0e3f4cea166a143596260e16ce2'},
 'version': '2.0',
 'id': 'e80cfb8241f45370d8df6caad9f531e2a3c061d7c816b4154f72f5429153441d'}

In [39]:
create_tx_info = db.transactions.retrieve(tx_reciept_id)
create_tx_info

{'inputs': [{'owners_before': ['EhBGDJnKjATfM1eRuMw7ZNDd7XSkhPLi7vw1fYDUgv5G'],
   'fulfills': None,
   'fulfillment': 'pGSAIMtyxBirvd3MiuxsEPaJXdTMlXwJsh2Fzw0JLOoi8BpzgUCbbE79y8Gyb4YxIydADhstC9OJaN5uSukSg1bSWx31NREqBkYHlIAcN7KIMp9y_yhsDlIvp4jnt8DUNEsS0kAD'}],
 'outputs': [{'public_keys': ['CfEpmfVLny134UvWczVux4j8td9srvyeYy7hPEZXT1jG'],
   'condition': {'details': {'type': 'ed25519-sha-256',
     'public_key': 'CfEpmfVLny134UvWczVux4j8td9srvyeYy7hPEZXT1jG'},
    'uri': 'ni:///sha-256;53p3B2t3tovqxEsX6x0rdPgJ1Rw7OFDE47EdTYf8fVg?fpt=ed25519-sha-256&cost=131072'},
   'amount': '10'}],
 'operation': 'CREATE',
 'metadata': {'data': 'DODO'},
 'asset': {'data': {'token_for': {'game_boy': {'serial_number': 'LR35902'}},
   'description': 'Time share token. Each token equals one hour of usage.'}},
 'version': '2.0',
 'id': '73a6e5517037673ca1bc49dc6df9c722b425e0e3f4cea166a143596260e16ce2'}

# Update 2

In [15]:
transfer_asset = {"id": fulfilled_token_tx["id"]}
output_index = 0
output = fulfilled_token_tx["outputs"][output_index]
transfer_input = {
    "fulfillment": output["condition"]["details"],
    "fulfills": {"output_index": output_index, "transaction_id": transfer_asset["id"]},
    "owners_before": output["public_keys"],
}

In [16]:
transfer_input

{'fulfillment': {'type': 'ed25519-sha-256',
  'public_key': '61zkK6phc2T4FbgeMekcffUttUCsvLtQXrFCdTAzBbvi'},
 'fulfills': {'output_index': 0,
  'transaction_id': '8d72512115f51a1a7dc0e7ff3a617e8f6c60c602f989c77228e3549c35e83e7d'},
 'owners_before': ['61zkK6phc2T4FbgeMekcffUttUCsvLtQXrFCdTAzBbvi']}

In [17]:
prepared_transfer_tx = db.transactions.prepare(
    operation="TRANSFER",
    asset=transfer_asset,
    inputs=transfer_input,
    metadata={"data":"8888888888"},
    recipients=[(
                [bob.public_key],
                 10
                 )],
)

In [18]:
prepared_transfer_tx

{'inputs': [{'owners_before': ['61zkK6phc2T4FbgeMekcffUttUCsvLtQXrFCdTAzBbvi'],
   'fulfills': {'transaction_id': '8d72512115f51a1a7dc0e7ff3a617e8f6c60c602f989c77228e3549c35e83e7d',
    'output_index': 0},
   'fulfillment': {'type': 'ed25519-sha-256',
    'public_key': '61zkK6phc2T4FbgeMekcffUttUCsvLtQXrFCdTAzBbvi'}}],
 'outputs': [{'public_keys': ['61zkK6phc2T4FbgeMekcffUttUCsvLtQXrFCdTAzBbvi'],
   'condition': {'details': {'type': 'ed25519-sha-256',
     'public_key': '61zkK6phc2T4FbgeMekcffUttUCsvLtQXrFCdTAzBbvi'},
    'uri': 'ni:///sha-256;hD0B4GxNCq4LMd_RR6qtoZwVFbCA4UAj48jmrWuPFi8?fpt=ed25519-sha-256&cost=131072'},
   'amount': '10'}],
 'operation': 'TRANSFER',
 'metadata': {'data': '8888888888'},
 'asset': {'id': '8d72512115f51a1a7dc0e7ff3a617e8f6c60c602f989c77228e3549c35e83e7d'},
 'version': '2.0',
 'id': None}

In [19]:
fulfilled_transfer_tx = db.transactions.fulfill(
    prepared_transfer_tx,
    private_keys=bob.private_key
)

HASH DATA <class 'str'>, data='{"asset":{"id":"8d72512115f51a1a7dc0e7ff3a617e8f6c60c602f989c77228e3549c35e83e7d"},"id":null,"inputs":[{"fulfillment":"pGSAIEqLuqZjLzeFvVas0AOLkCJyNdhQbc6H7atq-ouSiBMzgUCx58zTCYiSKYKvdRlsPmPL0t0aR4eflTyJPxOMqLaUnVgFrpUt8koGhj8DI_8TFj_fIO1MCJCLvO34z_b998MN","fulfills":{"output_index":0,"transaction_id":"8d72512115f51a1a7dc0e7ff3a617e8f6c60c602f989c77228e3549c35e83e7d"},"owners_before":["61zkK6phc2T4FbgeMekcffUttUCsvLtQXrFCdTAzBbvi"]}],"metadata":{"data":"8888888888"},"operation":"TRANSFER","outputs":[{"amount":"10","condition":{"details":{"public_key":"61zkK6phc2T4FbgeMekcffUttUCsvLtQXrFCdTAzBbvi","type":"ed25519-sha-256"},"uri":"ni:///sha-256;hD0B4GxNCq4LMd_RR6qtoZwVFbCA4UAj48jmrWuPFi8?fpt=ed25519-sha-256&cost=131072"},"public_keys":["61zkK6phc2T4FbgeMekcffUttUCsvLtQXrFCdTAzBbvi"]}],"version":"2.0"}'


In [20]:
fulfilled_transfer_tx

{'inputs': [{'owners_before': ['61zkK6phc2T4FbgeMekcffUttUCsvLtQXrFCdTAzBbvi'],
   'fulfills': {'transaction_id': '8d72512115f51a1a7dc0e7ff3a617e8f6c60c602f989c77228e3549c35e83e7d',
    'output_index': 0},
   'fulfillment': 'pGSAIEqLuqZjLzeFvVas0AOLkCJyNdhQbc6H7atq-ouSiBMzgUCx58zTCYiSKYKvdRlsPmPL0t0aR4eflTyJPxOMqLaUnVgFrpUt8koGhj8DI_8TFj_fIO1MCJCLvO34z_b998MN'}],
 'outputs': [{'public_keys': ['61zkK6phc2T4FbgeMekcffUttUCsvLtQXrFCdTAzBbvi'],
   'condition': {'details': {'type': 'ed25519-sha-256',
     'public_key': '61zkK6phc2T4FbgeMekcffUttUCsvLtQXrFCdTAzBbvi'},
    'uri': 'ni:///sha-256;hD0B4GxNCq4LMd_RR6qtoZwVFbCA4UAj48jmrWuPFi8?fpt=ed25519-sha-256&cost=131072'},
   'amount': '10'}],
 'operation': 'TRANSFER',
 'metadata': {'data': '8888888888'},
 'asset': {'id': '8d72512115f51a1a7dc0e7ff3a617e8f6c60c602f989c77228e3549c35e83e7d'},
 'version': '2.0',
 'id': 'fc941b54269f07bbc64176487df4ed6dc6dd0eee612e4932862ce9e576a01fed'}

In [21]:
transfer_tx = db.transactions.send_commit(fulfilled_transfer_tx)
transfer_tx_id = transfer_tx[4:]

In [22]:
transfer_tx_info = db.transactions.retrieve(transfer_tx_id)
transfer_tx_info

{'inputs': [{'owners_before': ['61zkK6phc2T4FbgeMekcffUttUCsvLtQXrFCdTAzBbvi'],
   'fulfills': {'transaction_id': '8d72512115f51a1a7dc0e7ff3a617e8f6c60c602f989c77228e3549c35e83e7d',
    'output_index': 0},
   'fulfillment': 'pGSAIEqLuqZjLzeFvVas0AOLkCJyNdhQbc6H7atq-ouSiBMzgUCx58zTCYiSKYKvdRlsPmPL0t0aR4eflTyJPxOMqLaUnVgFrpUt8koGhj8DI_8TFj_fIO1MCJCLvO34z_b998MN'}],
 'outputs': [{'public_keys': ['61zkK6phc2T4FbgeMekcffUttUCsvLtQXrFCdTAzBbvi'],
   'condition': {'details': {'type': 'ed25519-sha-256',
     'public_key': '61zkK6phc2T4FbgeMekcffUttUCsvLtQXrFCdTAzBbvi'},
    'uri': 'ni:///sha-256;hD0B4GxNCq4LMd_RR6qtoZwVFbCA4UAj48jmrWuPFi8?fpt=ed25519-sha-256&cost=131072'},
   'amount': '10'}],
 'operation': 'TRANSFER',
 'metadata': {'data': '8888888888'},
 'asset': {'id': '8d72512115f51a1a7dc0e7ff3a617e8f6c60c602f989c77228e3549c35e83e7d'},
 'version': '2.0',
 'id': 'fc941b54269f07bbc64176487df4ed6dc6dd0eee612e4932862ce9e576a01fed'}

In [23]:
nft_list_init_token = {
    "data": {
        "token_for": {"NFT_list": "123asd"},
        "description": "NFT List tracking token",
    },
}

nft_init_metadata = {
    "data": "INIT EMPTY HASH"
}

admin = generate_keypair()

prepared_token_tx = db.transactions.prepare(
    operation="CREATE",
    signers=admin.public_key,
    recipients=[([admin.public_key], 1)],
    asset=nft_list_init_token,
    metadata=nft_init_metadata
)

# fulfill the tnx
fulfilled_token_tx = db.transactions.fulfill(
    prepared_token_tx,
    private_keys=admin.private_key
)

tx_reciept = db.transactions.send_commit(fulfilled_token_tx)
tx_reciept_id = tx_reciept[4:]

print(tx_reciept_id)


HASH DATA <class 'str'>, data='{"asset":{"data":{"description":"NFT List tracking token","token_for":{"NFT_list":"123asd"}}},"id":null,"inputs":[{"fulfillment":"pGSAIDgs5lZ_iqENBRo5Yv3tvglhMgr-m181T5BKzB2JtkN5gUBMbekuppGdH0ujUrROP9QLuNv6cEwy2_hbLM7yKTrboOn_GRt2TO3k4F_FmD0YBWnq8Mt5j1kzlVbkPQZnvfEB","fulfills":null,"owners_before":["4nHYMjqyxKBuvPzUpQkParLrXTDvmCBsz6GpSECqH7eY"]}],"metadata":{"data":"INIT EMPTY HASH"},"operation":"CREATE","outputs":[{"amount":"1","condition":{"details":{"public_key":"4nHYMjqyxKBuvPzUpQkParLrXTDvmCBsz6GpSECqH7eY","type":"ed25519-sha-256"},"uri":"ni:///sha-256;nlzo8QML5wgHF8IkhEhHVlLHVxsPhTb_KFqGGl_qEUY?fpt=ed25519-sha-256&cost=131072"},"public_keys":["4nHYMjqyxKBuvPzUpQkParLrXTDvmCBsz6GpSECqH7eY"]}],"version":"2.0"}'
c31f7c35329bd88e371ab5437db3413216c5d2e9ba9d026d599b1c5e3ea3bf47


In [24]:
tx_reciept_id

'c31f7c35329bd88e371ab5437db3413216c5d2e9ba9d026d599b1c5e3ea3bf47'

In [25]:
transfer_tx_info = db.transactions.retrieve(tx_reciept_id)
transfer_tx_info


{'inputs': [{'owners_before': ['4nHYMjqyxKBuvPzUpQkParLrXTDvmCBsz6GpSECqH7eY'],
   'fulfills': None,
   'fulfillment': 'pGSAIDgs5lZ_iqENBRo5Yv3tvglhMgr-m181T5BKzB2JtkN5gUBMbekuppGdH0ujUrROP9QLuNv6cEwy2_hbLM7yKTrboOn_GRt2TO3k4F_FmD0YBWnq8Mt5j1kzlVbkPQZnvfEB'}],
 'outputs': [{'public_keys': ['4nHYMjqyxKBuvPzUpQkParLrXTDvmCBsz6GpSECqH7eY'],
   'condition': {'details': {'type': 'ed25519-sha-256',
     'public_key': '4nHYMjqyxKBuvPzUpQkParLrXTDvmCBsz6GpSECqH7eY'},
    'uri': 'ni:///sha-256;nlzo8QML5wgHF8IkhEhHVlLHVxsPhTb_KFqGGl_qEUY?fpt=ed25519-sha-256&cost=131072'},
   'amount': '1'}],
 'operation': 'CREATE',
 'metadata': {'data': 'INIT EMPTY HASH'},
 'asset': {'data': {'token_for': {'NFT_list': '123asd'},
   'description': 'NFT List tracking token'}},
 'version': '2.0',
 'id': 'c31f7c35329bd88e371ab5437db3413216c5d2e9ba9d026d599b1c5e3ea3bf47'}

# Dynamic Transfer by Admin

In [1]:
from resdb_driver.crypto import generate_keypair
from resdb_driver import Resdb

db_root_url = "http://0.0.0.0:18000"

db = Resdb(db_root_url)

admin=generate_keypair()

token_supply : int = 1

# create a digital asset for Alice
nft_list_init_token = {
    "data": {
        "token_for": {"NFT_list": "123asd"},
        "description": "NFT List tracking token",
    },
}

nft_init_metadata = {
        "data": "INIT EMPTY HASH"
    }

# prepare the transaction with the digital asset and issue 10 tokens for Bob
prepared_token_tx = db.transactions.prepare(
    operation="CREATE",
    signers=admin.public_key,
    recipients=[([admin.public_key], token_supply)],
    asset=game_boy_token,
    metadata=nft_init_metadata
)

# fulfill the tnx
fulfilled_token_tx = db.transactions.fulfill(
    prepared_token_tx,
    private_keys=admin.private_key)

NameError: name 'game_boy_token' is not defined

In [2]:
from resdb_driver.crypto import generate_keypair
from resdb_driver import Resdb

db_root_url = "http://0.0.0.0:18000"

db = Resdb(db_root_url)

admin=generate_keypair()

token_supply: int = 1

nft_list_init_token = {
    "data": {
        "token_for": {"NFT_list": "Used to store"},
        "description": "NFT List Token",
    },
}

nft_init_metadata = {
    "data": "INIT EMPTY HASH"
}

prepared_token_tx = db.transactions.prepare(
    operation="CREATE",
    signers=admin.public_key,
    recipients=[([admin.public_key], 1)],
    asset=nft_list_init_token,
    metadata={"data": "LIST INITIALISED"}
)

# fulfill the tnx
fulfilled_token_tx = db.transactions.fulfill(
    prepared_token_tx,
    private_keys=admin.private_key
)

HASH DATA <class 'str'>, data='{"asset":{"data":{"description":"NFT List Token","token_for":{"NFT_list":"Used to store"}}},"id":null,"inputs":[{"fulfillment":"pGSAIE7mv2xzqgqZIK_Act1QjvogLNEVflbARgl0qnWmdEJVgUDBP-Fn8NXRfzCkiO7s0_G69x6lCsoZhdTaBxmbySvNyCzcN4FFplqVKfwo7cf8khCB1Qg5OOJpN8kj0GedXIYE","fulfills":null,"owners_before":["6JzskBGQUxWmo6r5jR5CQjuQiyxgVF3ZNST1ciu1enXn"]}],"metadata":{"data":"LIST INITIALISED"},"operation":"CREATE","outputs":[{"amount":"1","condition":{"details":{"public_key":"6JzskBGQUxWmo6r5jR5CQjuQiyxgVF3ZNST1ciu1enXn","type":"ed25519-sha-256"},"uri":"ni:///sha-256;WvsSmyHCkjTZa8NRgWhIZVJmNsVQ8lXAQLVM2nNcr5M?fpt=ed25519-sha-256&cost=131072"},"public_keys":["6JzskBGQUxWmo6r5jR5CQjuQiyxgVF3ZNST1ciu1enXn"]}],"version":"2.0"}'


In [10]:
fulfilled_token_tx

{'inputs': [{'owners_before': ['EA5PttQRPtNuKqz1A1N4eo7DjVGtmavFjkW7A8L2JDS2'],
   'fulfills': None,
   'fulfillment': 'pGSAIMN7IGAWwyAV5k5C0FlpmqzMkH9DGNF2PnWMn1T53uYzgUBDkArUlU6gv9g3KkoA6xbxEpmz8zUwmYp35y1NiMRNUg0djBLF50IR5fwwtRaGNhpHrZxuBSteTizc7C7prPkL'}],
 'outputs': [{'public_keys': ['EA5PttQRPtNuKqz1A1N4eo7DjVGtmavFjkW7A8L2JDS2'],
   'condition': {'details': {'type': 'ed25519-sha-256',
     'public_key': 'EA5PttQRPtNuKqz1A1N4eo7DjVGtmavFjkW7A8L2JDS2'},
    'uri': 'ni:///sha-256;wnt2eydBsbcx6HVGeohta8RXKhzCfkjbjYk1bqh4WJ0?fpt=ed25519-sha-256&cost=131072'},
   'amount': '1'}],
 'operation': 'CREATE',
 'metadata': {'data': 'LIST INITIALISED'},
 'asset': {'data': {'token_for': {'NFT_list': 'Used to store'},
   'description': 'NFT List Token'}},
 'version': '2.0',
 'id': '264b88bf9358ccd3bcdba15186d04c50f366563a46158550d9258dca5bc43e44'}

In [11]:
tx_reciept = db.transactions.send_commit(fulfilled_token_tx)
tx_reciept_id = tx_reciept[4:]

In [13]:
create_tx_info = db.transactions.retrieve(tx_reciept_id)
create_tx_info

{'inputs': [{'owners_before': ['EA5PttQRPtNuKqz1A1N4eo7DjVGtmavFjkW7A8L2JDS2'],
   'fulfills': None,
   'fulfillment': 'pGSAIMN7IGAWwyAV5k5C0FlpmqzMkH9DGNF2PnWMn1T53uYzgUBDkArUlU6gv9g3KkoA6xbxEpmz8zUwmYp35y1NiMRNUg0djBLF50IR5fwwtRaGNhpHrZxuBSteTizc7C7prPkL'}],
 'outputs': [{'public_keys': ['EA5PttQRPtNuKqz1A1N4eo7DjVGtmavFjkW7A8L2JDS2'],
   'condition': {'details': {'type': 'ed25519-sha-256',
     'public_key': 'EA5PttQRPtNuKqz1A1N4eo7DjVGtmavFjkW7A8L2JDS2'},
    'uri': 'ni:///sha-256;wnt2eydBsbcx6HVGeohta8RXKhzCfkjbjYk1bqh4WJ0?fpt=ed25519-sha-256&cost=131072'},
   'amount': '1'}],
 'operation': 'CREATE',
 'metadata': {'data': 'LIST INITIALISED'},
 'asset': {'data': {'token_for': {'NFT_list': 'Used to store'},
   'description': 'NFT List Token'}},
 'version': '2.0',
 'id': '264b88bf9358ccd3bcdba15186d04c50f366563a46158550d9258dca5bc43e44'}

In [14]:
create_tx_info["id"]

'264b88bf9358ccd3bcdba15186d04c50f366563a46158550d9258dca5bc43e44'

In [15]:
# Dynamic
transfer_asset = {"id": create_tx_info["id"]}

output_index = 0

output = create_tx_info["outputs"][output_index]

transfer_input = {
    "fulfillment": output["condition"]["details"],
    "fulfills": {"output_index": output_index, "transaction_id": transfer_asset["id"]},
    "owners_before": output["public_keys"],
}


In [16]:
# Dynamic

prepared_transfer_tx = db.transactions.prepare(
    operation="TRANSFER",
    asset=transfer_asset,
    inputs=transfer_input,
    metadata={"data": ["123123sadfsafasd","sdafasdfasdfasdf54631685468546","fsADFASDF654654654654654"]},
    recipients=[(
                [admin.public_key],
                1
                )],
)

In [17]:
# Dynamic
fulfilled_transfer_tx = db.transactions.fulfill(
    prepared_transfer_tx,
    private_keys=admin.private_key
)


HASH DATA <class 'str'>, data='{"asset":{"id":"264b88bf9358ccd3bcdba15186d04c50f366563a46158550d9258dca5bc43e44"},"id":null,"inputs":[{"fulfillment":"pGSAIMN7IGAWwyAV5k5C0FlpmqzMkH9DGNF2PnWMn1T53uYzgUCcxOrEV2_HC0yd0W-8qmcqCmc25gcj65KYsqv0u2ovTkqR-eDwlBxO5YwI5DtZqRgxO2BwA7M5lBl0kGSY_gAH","fulfills":{"output_index":0,"transaction_id":"264b88bf9358ccd3bcdba15186d04c50f366563a46158550d9258dca5bc43e44"},"owners_before":["EA5PttQRPtNuKqz1A1N4eo7DjVGtmavFjkW7A8L2JDS2"]}],"metadata":{"data":["123123sadfsafasd","sdafasdfasdfasdf54631685468546","fsADFASDF654654654654654"]},"operation":"TRANSFER","outputs":[{"amount":"1","condition":{"details":{"public_key":"EA5PttQRPtNuKqz1A1N4eo7DjVGtmavFjkW7A8L2JDS2","type":"ed25519-sha-256"},"uri":"ni:///sha-256;wnt2eydBsbcx6HVGeohta8RXKhzCfkjbjYk1bqh4WJ0?fpt=ed25519-sha-256&cost=131072"},"public_keys":["EA5PttQRPtNuKqz1A1N4eo7DjVGtmavFjkW7A8L2JDS2"]}],"version":"2.0"}'


In [18]:
# Dynamic
transfer_tx = db.transactions.send_commit(fulfilled_transfer_tx)
transfer_tx_id = transfer_tx[4:]

In [19]:
# Dynamic
transfer_tx_info = db.transactions.retrieve(transfer_tx_id)
transfer_tx_info


{'inputs': [{'owners_before': ['EA5PttQRPtNuKqz1A1N4eo7DjVGtmavFjkW7A8L2JDS2'],
   'fulfills': {'transaction_id': '264b88bf9358ccd3bcdba15186d04c50f366563a46158550d9258dca5bc43e44',
    'output_index': 0},
   'fulfillment': 'pGSAIMN7IGAWwyAV5k5C0FlpmqzMkH9DGNF2PnWMn1T53uYzgUCcxOrEV2_HC0yd0W-8qmcqCmc25gcj65KYsqv0u2ovTkqR-eDwlBxO5YwI5DtZqRgxO2BwA7M5lBl0kGSY_gAH'}],
 'outputs': [{'public_keys': ['EA5PttQRPtNuKqz1A1N4eo7DjVGtmavFjkW7A8L2JDS2'],
   'condition': {'details': {'type': 'ed25519-sha-256',
     'public_key': 'EA5PttQRPtNuKqz1A1N4eo7DjVGtmavFjkW7A8L2JDS2'},
    'uri': 'ni:///sha-256;wnt2eydBsbcx6HVGeohta8RXKhzCfkjbjYk1bqh4WJ0?fpt=ed25519-sha-256&cost=131072'},
   'amount': '1'}],
 'operation': 'TRANSFER',
 'metadata': {'data': ['123123sadfsafasd',
   'sdafasdfasdfasdf54631685468546',
   'fsADFASDF654654654654654']},
 'asset': {'id': '264b88bf9358ccd3bcdba15186d04c50f366563a46158550d9258dca5bc43e44'},
 'version': '2.0',
 'id': '54fca8798402114013998fa2b12b45ae4ccc2b6abcd23a5f83132f

### Transfer from transfer: Update Metadata

In [21]:
transfer_tx_info = db.transactions.retrieve(transfer_tx_id)
transfer_tx_info

{'inputs': [{'owners_before': ['EA5PttQRPtNuKqz1A1N4eo7DjVGtmavFjkW7A8L2JDS2'],
   'fulfills': {'transaction_id': '264b88bf9358ccd3bcdba15186d04c50f366563a46158550d9258dca5bc43e44',
    'output_index': 0},
   'fulfillment': 'pGSAIMN7IGAWwyAV5k5C0FlpmqzMkH9DGNF2PnWMn1T53uYzgUCcxOrEV2_HC0yd0W-8qmcqCmc25gcj65KYsqv0u2ovTkqR-eDwlBxO5YwI5DtZqRgxO2BwA7M5lBl0kGSY_gAH'}],
 'outputs': [{'public_keys': ['EA5PttQRPtNuKqz1A1N4eo7DjVGtmavFjkW7A8L2JDS2'],
   'condition': {'details': {'type': 'ed25519-sha-256',
     'public_key': 'EA5PttQRPtNuKqz1A1N4eo7DjVGtmavFjkW7A8L2JDS2'},
    'uri': 'ni:///sha-256;wnt2eydBsbcx6HVGeohta8RXKhzCfkjbjYk1bqh4WJ0?fpt=ed25519-sha-256&cost=131072'},
   'amount': '1'}],
 'operation': 'TRANSFER',
 'metadata': {'data': ['123123sadfsafasd',
   'sdafasdfasdfasdf54631685468546',
   'fsADFASDF654654654654654']},
 'asset': {'id': '264b88bf9358ccd3bcdba15186d04c50f366563a46158550d9258dca5bc43e44'},
 'version': '2.0',
 'id': '54fca8798402114013998fa2b12b45ae4ccc2b6abcd23a5f83132f

In [22]:
transfer_tx_info["asset"]["id"]

'264b88bf9358ccd3bcdba15186d04c50f366563a46158550d9258dca5bc43e44'

In [23]:
transfer_tx_info["outputs"][output_index]

{'public_keys': ['EA5PttQRPtNuKqz1A1N4eo7DjVGtmavFjkW7A8L2JDS2'],
 'condition': {'details': {'type': 'ed25519-sha-256',
   'public_key': 'EA5PttQRPtNuKqz1A1N4eo7DjVGtmavFjkW7A8L2JDS2'},
  'uri': 'ni:///sha-256;wnt2eydBsbcx6HVGeohta8RXKhzCfkjbjYk1bqh4WJ0?fpt=ed25519-sha-256&cost=131072'},
 'amount': '1'}

In [24]:
transfer_asset = {"id": transfer_tx_info["asset"]["id"]}

output_index = 0

output = transfer_tx_info["outputs"][output_index]

transfer_input = {
    "fulfillment": output["condition"]["details"],
    "fulfills": {"output_index": output_index, "transaction_id": transfer_asset["id"]},
    "owners_before": output["public_keys"],
}

In [25]:
transfer_tx_info["metadata"]

{'data': ['123123sadfsafasd',
  'sdafasdfasdfasdf54631685468546',
  'fsADFASDF654654654654654']}

In [26]:
old_metadata = transfer_tx_info["metadata"]["data"]

In [27]:
import copy

new_metadata = copy.deepcopy(old_metadata)

In [28]:
new_list_element = "SHIVAM"

In [29]:
new_metadata.append(new_list_element)

In [30]:
new_metadata

['123123sadfsafasd',
 'sdafasdfasdfasdf54631685468546',
 'fsADFASDF654654654654654',
 'SHIVAM']

In [31]:
# print(new_metadata,id(new_metadata),id(old_metadata))

In [32]:
# Dynamic

prepared_transfer_tx = db.transactions.prepare(
    operation="TRANSFER",
    asset=transfer_asset,
    inputs=transfer_input,
    metadata={"data": new_metadata},
    recipients=[(
                [admin.public_key],
                1
                )],
)

In [33]:
# Dynamic
fulfilled_transfer_tx = db.transactions.fulfill(
    prepared_transfer_tx,
    private_keys=admin.private_key
)

HASH DATA <class 'str'>, data='{"asset":{"id":"264b88bf9358ccd3bcdba15186d04c50f366563a46158550d9258dca5bc43e44"},"id":null,"inputs":[{"fulfillment":"pGSAIMN7IGAWwyAV5k5C0FlpmqzMkH9DGNF2PnWMn1T53uYzgUB1ve3mftKW1SykSYUI4cO1pLOzWu5AL2VHkgeASM9NsT2hec_e7kDxkaUUufUobd1Xw3hasMlCiWNB8H6i6ucM","fulfills":{"output_index":0,"transaction_id":"264b88bf9358ccd3bcdba15186d04c50f366563a46158550d9258dca5bc43e44"},"owners_before":["EA5PttQRPtNuKqz1A1N4eo7DjVGtmavFjkW7A8L2JDS2"]}],"metadata":{"data":["123123sadfsafasd","sdafasdfasdfasdf54631685468546","fsADFASDF654654654654654","SHIVAM"]},"operation":"TRANSFER","outputs":[{"amount":"1","condition":{"details":{"public_key":"EA5PttQRPtNuKqz1A1N4eo7DjVGtmavFjkW7A8L2JDS2","type":"ed25519-sha-256"},"uri":"ni:///sha-256;wnt2eydBsbcx6HVGeohta8RXKhzCfkjbjYk1bqh4WJ0?fpt=ed25519-sha-256&cost=131072"},"public_keys":["EA5PttQRPtNuKqz1A1N4eo7DjVGtmavFjkW7A8L2JDS2"]}],"version":"2.0"}'


In [34]:
# Dynamic
transfer_tx = db.transactions.send_commit(fulfilled_transfer_tx)
transfer_tx_id = transfer_tx[4:]


In [35]:
transfer_tx_info = db.transactions.retrieve(transfer_tx_id)
transfer_tx_info


{'inputs': [{'owners_before': ['EA5PttQRPtNuKqz1A1N4eo7DjVGtmavFjkW7A8L2JDS2'],
   'fulfills': {'transaction_id': '264b88bf9358ccd3bcdba15186d04c50f366563a46158550d9258dca5bc43e44',
    'output_index': 0},
   'fulfillment': 'pGSAIMN7IGAWwyAV5k5C0FlpmqzMkH9DGNF2PnWMn1T53uYzgUB1ve3mftKW1SykSYUI4cO1pLOzWu5AL2VHkgeASM9NsT2hec_e7kDxkaUUufUobd1Xw3hasMlCiWNB8H6i6ucM'}],
 'outputs': [{'public_keys': ['EA5PttQRPtNuKqz1A1N4eo7DjVGtmavFjkW7A8L2JDS2'],
   'condition': {'details': {'type': 'ed25519-sha-256',
     'public_key': 'EA5PttQRPtNuKqz1A1N4eo7DjVGtmavFjkW7A8L2JDS2'},
    'uri': 'ni:///sha-256;wnt2eydBsbcx6HVGeohta8RXKhzCfkjbjYk1bqh4WJ0?fpt=ed25519-sha-256&cost=131072'},
   'amount': '1'}],
 'operation': 'TRANSFER',
 'metadata': {'data': ['123123sadfsafasd',
   'sdafasdfasdfasdf54631685468546',
   'fsADFASDF654654654654654',
   'SHIVAM']},
 'asset': {'id': '264b88bf9358ccd3bcdba15186d04c50f366563a46158550d9258dca5bc43e44'},
 'version': '2.0',
 'id': 'b931351b27b523fe01ef00475287e3f625355c5e4

In [3]:
transfer_tx_id= "9e624e14306918a1faf292272ce03733b101eb16bb52662148dc25dc7619c43f"


In [4]:
transfer_tx_info = db.transactions.retrieve(transfer_tx_id)
transfer_tx_info


{'inputs': [{'owners_before': ['Gp3dcQddpwTo5sk48PsxpWwsi2A7aXbCFCVpkoJKLWd4'],
   'fulfills': None,
   'fulfillment': 'pGSAIOrsk7lMDZgOtbxU6FiTrZ_l6qQcBqSRNQTnkNAxhGb3gUAgJM_fJCY_cf_H9bGXE74QQgGYJNgv56CjOfhmctoT54csC1H-A_Tz_LZMRbIVCB2kKiBmcBUH25bFZtRINP0A'}],
 'outputs': [{'public_keys': ['APQyo9isPTztNh87f7J1tSM2fABmN3Ub6XnE7ZCX6YFx'],
   'condition': {'details': {'type': 'ed25519-sha-256',
     'public_key': 'APQyo9isPTztNh87f7J1tSM2fABmN3Ub6XnE7ZCX6YFx'},
    'uri': 'ni:///sha-256;MoaT_HCP7eDiKU23tZCiwBo3OMqYod8v5jth706Bbdg?fpt=ed25519-sha-256&cost=131072'},
   'amount': '10'}],
 'operation': 'CREATE',
 'metadata': None,
 'asset': {'data': {'nft_name': 'Monkey Dance',
   'ipfs_file_url': 'https://www.youtube.com/watch?v=opyPkipNNhE',
   'creator_name': 'KoKo',
   'supply': 10,
   'description': 'New Monkey Videos',
   'cost': 5.0,
   'currency': 'BTC',
   'creator_external_link': 'https://github.com/kaustubhKVS',
   'view': 2,
   'tx_id': 'NULL'}},
 'version': '2.0',
 'id': '9e624e

In [29]:
transfer_asset = {"id": fulfilled_token_tx["id"]}
output_index = 0
output = fulfilled_token_tx["outputs"][output_index]
transfer_input = {
    "fulfillment": output["condition"]["details"],
    "fulfills": {"output_index": output_index, "transaction_id": transfer_asset["id"]},
    "owners_before": output["public_keys"],
}

prepared_transfer_tx = db.transactions.prepare(
    operation="TRANSFER",
    asset=transfer_asset,
    inputs=transfer_input,
    metadata={"data": "2323"},
    recipients=[(
                [bob.public_key],
                10
                )],
)

fulfilled_transfer_tx = db.transactions.fulfill(
    prepared_transfer_tx,
    private_keys=bob.private_key
)

HASH DATA <class 'str'>, data='{"asset":{"id":"d8d519fff3fc5cc12eb4603b2f7ca85e6311328bdf3ea0de954d61c918fd3ea5"},"id":null,"inputs":[{"fulfillment":"pGSAIEsJeuVtxTgKrnL0EeYMwP4reMiKtfzJTPGuQPuGTr_EgUAErQZGIZeH-mM_W6EFeu1JkKWE4h7_RQLpB2ZwP2jxXnfeSgHhLzGklwTKbnv0xDWkEhlr1sP7kgrLNAFWQ-oC","fulfills":{"output_index":0,"transaction_id":"d8d519fff3fc5cc12eb4603b2f7ca85e6311328bdf3ea0de954d61c918fd3ea5"},"owners_before":["63uxmeafqCP5T9umrBehzyK7gVuiCYR62HgXmUdDg5PZ"]}],"metadata":{"data":"2323"},"operation":"TRANSFER","outputs":[{"amount":"10","condition":{"details":{"public_key":"63uxmeafqCP5T9umrBehzyK7gVuiCYR62HgXmUdDg5PZ","type":"ed25519-sha-256"},"uri":"ni:///sha-256;HxreroHLMj0S-ObK1DxzY02uWnQe3jmi62p_poQ-Iys?fpt=ed25519-sha-256&cost=131072"},"public_keys":["63uxmeafqCP5T9umrBehzyK7gVuiCYR62HgXmUdDg5PZ"]}],"version":"2.0"}'


In [30]:
transfer_tx = db.transactions.send_commit(fulfilled_transfer_tx)
transfer_tx_id = transfer_tx[4:]


In [31]:
transfer_tx_info = db.transactions.retrieve(transfer_tx_id)
transfer_tx_info


{'inputs': [{'owners_before': ['63uxmeafqCP5T9umrBehzyK7gVuiCYR62HgXmUdDg5PZ'],
   'fulfills': {'transaction_id': 'd8d519fff3fc5cc12eb4603b2f7ca85e6311328bdf3ea0de954d61c918fd3ea5',
    'output_index': 0},
   'fulfillment': 'pGSAIEsJeuVtxTgKrnL0EeYMwP4reMiKtfzJTPGuQPuGTr_EgUAErQZGIZeH-mM_W6EFeu1JkKWE4h7_RQLpB2ZwP2jxXnfeSgHhLzGklwTKbnv0xDWkEhlr1sP7kgrLNAFWQ-oC'}],
 'outputs': [{'public_keys': ['63uxmeafqCP5T9umrBehzyK7gVuiCYR62HgXmUdDg5PZ'],
   'condition': {'details': {'type': 'ed25519-sha-256',
     'public_key': '63uxmeafqCP5T9umrBehzyK7gVuiCYR62HgXmUdDg5PZ'},
    'uri': 'ni:///sha-256;HxreroHLMj0S-ObK1DxzY02uWnQe3jmi62p_poQ-Iys?fpt=ed25519-sha-256&cost=131072'},
   'amount': '10'}],
 'operation': 'TRANSFER',
 'metadata': {'data': '2323'},
 'asset': {'id': 'd8d519fff3fc5cc12eb4603b2f7ca85e6311328bdf3ea0de954d61c918fd3ea5'},
 'version': '2.0',
 'id': '3dd9003581695045775a9cc8701ec5ba718249d86d4aa45e2f7346fdd4305798'}

In [1]:
tx = {
    "nft_name": "Monkey Dance",
    "ipfs_file_url": "https://www.youtube.com/watch?v=opyPkipNNhE",
    "creator_name": "KoKo",
    "creator_external_link": "https://github.com/kaustubhKVS",
    "supply": 10,
    "description": "New Monkey Videos",
    "cost": 5,
    "currency": "BTC",
    "view": 2,
    "tx_id": "NULL"
}


In [2]:
tx["tx_id"] = "adsdasdasdasdasdasdasdasdasdasdasdasd"

In [3]:
tx

{'nft_name': 'Monkey Dance',
 'ipfs_file_url': 'https://www.youtube.com/watch?v=opyPkipNNhE',
 'creator_name': 'KoKo',
 'creator_external_link': 'https://github.com/kaustubhKVS',
 'supply': 10,
 'description': 'New Monkey Videos',
 'cost': 5,
 'currency': 'BTC',
 'view': 2,
 'tx_id': 'adsdasdasdasdasdasdasdasdasdasdasdasd'}